# Dataset & Batcher

在圖形處理中最常遇到的問題就是記憶體爆掉，以前我們可以一次將所有資料一口氣載入記憶體中進行處理，但是在一個圖片1mb的情況下只要1000張就需要占用1G的記憶體，更別提資料筆數都是萬筆起跳，而一張4萬的RTX2080TI也不過11GB記憶體，如何做記憶體控制就是圖項資料預處理中最重要的工作。



# ImageDataset

我們都知道只要讀入圖像就要吃記憶體，但是我們在訓練的時候每次只有batch size大小資料會參與訓練，所以我們可以設計一個工具平常時以圖像路徑的方式保存資料，只有在把資料取出的時候才轉換成圖像格式，如此一來我們就可以大幅度的減少記憶體的佔用，這就是ImageDataset的功用。

## parameter:

* X: 圖片的路徑
* Y: 圖片的標籤
* transformer: 對圖片處理的轉換function


# Batcher

如果說Dataset是物質轉化器，那麼Batcher就是分堆器。**在keras的model.fit中x參數給的資料型別如果有繼承Sequence 或 Generator型別就可以使用迭代的方式進行訓練**。

也就是接收一批批資料進行訓練，而不是一口氣讀入後在function內部自行分配成批次大小進行訓練。這樣的好處除了省記憶體外，我們可以透過給予transformer的方式每次對資料進行微調，幫助模型加速泛化減少過擬合。


## parameter:

* dataset: 資料集
* batch size: batch大小
* shuffle: 是否洗牌打亂

In [1]:
import math
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence



class ImageDataset:
    def __init__(self, X, Y, transformer = lambda x:x):
        self.X = X
        self.Y = Y
        self.transformer = transformer
        
    def __len__(self):
        return len(self.Y)
    def __getitem__(self, idx):
        
        # 將索引值對應的 image path取出
        if isinstance(idx, int) or isinstance(idx, slice):
            imgs = self.X[idx]
            labels = self.Y[idx]
        else:
            imgs = [self.X[i] for i in idx]
            labels = [self.Y[i] for i in idx]
        
        
        # 將 image path 轉為 np.array格式並處理
        if isinstance(imgs, list):
            imgs = [cv2.imread(img) for img in imgs] # 讀取
            imgs = [self.transformer(img) for img in imgs] # 轉換處理
            imgs = [np.expand_dims(img, axis=0) for img in imgs] # shape: (h, w, 3)->(1, h, w, 3)
            imgs = np.concatenate(imgs, axis=0) # 合併所有圖檔 shape變成: (batch size, h, w, 3)
        else:
            # 單張簡單處理
            imgs = cv2.imread(imgs)
            imgs = self.transformer(imgs)
        
        return np.array(imgs), np.array(labels)

    
    
    

class Batcher(Sequence):
    def __init__(self, datasets, batch_size, shuffle=False):
        self.datasets = datasets
        self.batch_size = batch_size
        self.indices = np.arange(0, len(self.datasets))
        if shuffle:
            np.random.shuffle(self.indices)
    def __len__(self):
        return math.ceil(len(self.datasets)/self.batch_size)
    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size: (idx+1)*self.batch_size]
        return self.datasets[indices]
        
        

In [2]:
def one_hot(arr, n):
    
    m = np.zeros((len(arr), n))
    
    for i, e in enumerate(arr):
        m[i][e] = 1
    return m

In [3]:
# 讀取資料改成ImageDataset格式

import os



db = 'datasets/17_flowers/'
X = []
Y = []

for dirs in os.listdir(db):
    if os.path.isdir(db+dirs+'/'):
        label = dirs
        for img in os.listdir(db+dirs+'/'):
            X.append(db+dirs+'/'+img)
            Y.append(int(label))
            
            
transformer = lambda x:cv2.resize(x, (128, 128))

datasets = ImageDataset(X, one_hot(Y, 17), transformer)

imgs, labels = datasets[0:3]
print(imgs.shape)
print(labels)
        


(3, 128, 128, 3)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [4]:
# 建立模型

from tensorflow.keras import models, layers


net = models.Sequential()

net.add(layers.Conv2D(10, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(128,128,3)))
net.add(layers.MaxPooling2D(pool_size=2))
net.add(layers.Conv2D(20, kernel_size=(3, 3), padding='same', activation='relu'))
net.add(layers.MaxPooling2D(pool_size=2))
net.add(layers.Conv2D(40, kernel_size=(3, 3), padding='same', activation='relu'))
net.add(layers.MaxPooling2D(pool_size=2))
net.add(layers.Flatten())
net.add(layers.Dense(17, activation='softmax'))

net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 10)      280       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 20)        1820      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 40)        7240      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 40)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10240)             0

In [5]:
# 開始訓練

from tensorflow.keras import metrics

batcher = Batcher(datasets, 20, shuffle=True)
net.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])
net.fit(x=batcher, epochs=10)

  ...
    to  
  ['...']
Train for 68 steps
Epoch 1/10
68/68 [==============================] - 4s 58ms/step - loss: 16.2387 - categorical_accuracy: 0.2544
Epoch 2/10
68/68 [==============================] - 4s 53ms/step - loss: 1.3392 - categorical_accuracy: 0.5831
Epoch 3/10
68/68 [==============================] - 4s 52ms/step - loss: 0.5822 - categorical_accuracy: 0.8346
Epoch 4/10
68/68 [==============================] - 4s 53ms/step - loss: 0.3556 - categorical_accuracy: 0.9279
Epoch 5/10
68/68 [==============================] - 4s 53ms/step - loss: 0.1384 - categorical_accuracy: 0.9662
Epoch 6/10
68/68 [==============================] - 4s 53ms/step - loss: 0.2351 - categorical_accuracy: 0.9706
Epoch 7/10
68/68 [==============================] - 4s 53ms/step - loss: 0.3443 - categorical_accuracy: 0.9691
Epoch 8/10
68/68 [==============================] - 4s 53ms/step - loss: 0.2783 - categorical_accuracy: 0.9647
Epoch 9/10
68/68 [==============================] - 4s 53ms/step - 